## Train a character-level GPT on some text data

The inputs here are simple text files, which we chop up to individual characters and then train GPT on. So you could say this is a char-transformer instead of a char-rnn. Doesn't quite roll off the tongue as well. In this example we will feed it some Shakespeare, which we'll get it to predict character-level.

In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt_tf.utils import set_seed
set_seed(42)

In [26]:
import numpy as np
import tensorflow as tf
from fastprogress import master_bar, progress_bar

In [4]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = tf.convert_to_tensor(dix[:-1], dtype=tf.int32)
        y = tf.convert_to_tensor(dix[1:], dtype=tf.int32)
        return x, y
    
    def gen(self):
        for idx in range(len(self)):
            # grab a chunk of (block_size + 1) characters from the data
            chunk = self.data[idx:idx + self.block_size + 1]
            # encode every character to an integer
            dix = [self.stoi[s] for s in chunk]
            x = tf.convert_to_tensor(dix[:-1], dtype=tf.int32)
            y = tf.convert_to_tensor(dix[1:], dtype=tf.int32)
            yield x, y


In [33]:
block_size = 128 # spatial extent of the model for its context

In [34]:
# you can download this file at https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt
text = open('input.txt', 'r').read() # don't worry we won't run out of file handles
train_dataset = CharDataset(text, block_size) # one line of poem is roughly 50 characters

data has 1115393 characters, 65 unique.


In [35]:
batch_size = 8
train_ds = tf.data.Dataset.from_generator(train_dataset.gen,
                                              (tf.int32, tf.int32),
                                              (tf.TensorShape([block_size]), tf.TensorShape([block_size]))
                                              ).batch(batch_size)

In [36]:
next(iter(train_ds))

(<tf.Tensor: shape=(8, 128), dtype=int32, numpy=
 array([[18, 47, 56, ..., 58, 53,  1],
        [47, 56, 57, ..., 53,  1, 42],
        [56, 57, 58, ...,  1, 42, 47],
        ...,
        [ 1, 15, 47, ..., 43,  1, 58],
        [15, 47, 58, ...,  1, 58, 46],
        [47, 58, 47, ..., 58, 46, 39]], dtype=int32)>,
 <tf.Tensor: shape=(8, 128), dtype=int32, numpy=
 array([[47, 56, 57, ..., 53,  1, 42],
        [56, 57, 58, ...,  1, 42, 47],
        [57, 58,  1, ..., 42, 47, 43],
        ...,
        [15, 47, 58, ...,  1, 58, 46],
        [47, 58, 47, ..., 58, 46, 39],
        [58, 47, 64, ..., 46, 39, 52]], dtype=int32)>)

In [39]:
from mingpt_tf.model import GPT, GPTConfig, TrainerConfig
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=4//2, n_head=4//2, n_embd=128//2)
tconf = TrainerConfig()
model = GPT(mconf, tconf)

In [38]:
# num_train_samples = 10000
# num_test_samples = 1000
# epochs = 100
# model.fit(train_ds
#         , steps_per_epoch=num_train_samples // batch_size                        
# #         , validation_data=test_dataset
# #         , validation_steps=num_test_samples // batch_size
# #         , batch_size=batch_size
#         , epochs=epochs
#         , shuffle=False)

Epoch 1/100


ValueError: in user code:

    /Users/tfrizza/Documents/robotic_learning/rl_venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /Users/tfrizza/Documents/robotic_learning/rl_venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/tfrizza/Documents/robotic_learning/rl_venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/tfrizza/Documents/robotic_learning/rl_venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/tfrizza/Documents/robotic_learning/rl_venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:533 train_step  **
  

In [40]:
cce_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
train_loss_metric = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)

def train_step(x,y):

    with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        logits = model(x,training=True)
        num_labels = tf.shape(logits)[-1]
        label_mask = tf.math.logical_not(y < 0)
        label_mask = tf.reshape(label_mask,(-1,))
        logits = tf.reshape(logits,(-1,num_labels))
        logits_masked = tf.boolean_mask(logits,label_mask)
        label_ids = tf.reshape(y,(-1,))
        label_ids_masked = tf.boolean_mask(label_ids,label_mask)
        loss = cce_loss(label_ids_masked, logits_masked)

    grads = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
    return loss

In [43]:
epoch_bar = master_bar(range(1))

for epoch in epoch_bar:
    for x,y in progress_bar(train_ds, total=len(train_dataset)//batch_size,parent=epoch_bar):
        loss = train_step(x,y)
#         self.tokens += tf.reduce_sum(tf.cast(inputs[1]>=0,tf.int32)).numpy()
        train_loss_metric(loss)
        epoch_bar.child.comment = f'training loss : {train_loss_metric.result()}'
    print(f"epoch {epoch+1}: train loss {train_loss_metric.result():.5f}.")
    train_loss_metric.reset_states()

KeyboardInterrupt: 

In [21]:
from mingpt_tf.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=2, batch_size=512, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                      num_workers=4)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()

epoch 1 iter 8: train loss 3.39783. lr 5.999939e-04:   0%|          | 9/2179 [00:50<3:24:14,  5.65s/it]


KeyboardInterrupt: 

In [44]:
# alright, let's sample some character-level Shakespeare
from mingpt_tf.utils import sample

context = "O God, O God!"
x = tf.convert_to_tensor([train_dataset.stoi[s] for s in context], dtype=tf.int32)[None,...]#.to(trainer.device)
y = sample(model, x, 20, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

O God, O God! deser tour, d y tin


In [ ]:
# well that was fun